In [ ]:
import os
import torch
from torchvision import models, transforms
from PIL import Image
from torch.nn.functional import normalize
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Load model once globally
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # Remove last fully connected layer
model.eval()

def get_embedding(imagem_path):
    try:
        preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        img = Image.open(imagem_path).convert('RGB')
        img_tensor = preprocess(img).unsqueeze(0)
        with torch.no_grad():
            embedding = model(img_tensor)
        embedding = normalize(embedding, p=2, dim=1)

        return embedding.squeeze(), img
    except Exception as e:
        print(f"Erro ao processar a imagem {imagem_path}: {e}")
        return None, None

def comparar_com_base(imagem_usuario_path, base_dir, top_k=5):
    print(f"Diretório de trabalho atual: {os.getcwd()}")
    base_dir = os.path.abspath(base_dir)
    print(f"Usando diretório base: {base_dir}")
    print(f"Extraindo embedding da imagem do usuário: {imagem_usuario_path}")
    emb_usuario, img_usuario = get_embedding(imagem_usuario_path)
    if emb_usuario is None:
        print("❌ Não foi possível extrair embedding da imagem fornecida.")
        return

    resultados = []
    total_files = 0
    processed_files = 0
    for root, _, files in os.walk(base_dir):
        for fname in files:
            if not fname.lower().endswith(('jpg', 'jpeg', 'png', 'bmp')):
                continue
            total_files += 1

    print(f"Total de imagens na base: {total_files}")

    for root, _, files in os.walk(base_dir):
        for fname in files:
            if not fname.lower().endswith(('jpg', 'jpeg', 'png', 'bmp')):
                continue
            img_path = os.path.join(root, fname)
            emb_base, _ = get_embedding(img_path)
            if emb_base is None:
                continue
            dist = 1 - torch.cosine_similarity(emb_usuario, emb_base, dim=0).item()
            class_name = os.path.basename(os.path.dirname(img_path))
            resultados.append((img_path, dist, class_name))
            processed_files += 1
            if processed_files % 10 == 0:
                print(f"Processadas {processed_files} imagens...")

    resultados.sort(key=lambda x: x[1])
    top_resultados = resultados[:top_k]

    plt.imshow(img_usuario)
    plt.title("Imagem do usuário")
    plt.axis('off')
    plt.show()

    for path, dist, class_name in top_resultados:
        _, img = get_embedding(path)
        plt.imshow(img)
        plt.title(f"Classe: {class_name} | Distância: {dist:.4f}")
        plt.axis('off')
        plt.show()
        print(f"📁 Classe: {class_name} | 📄 Imagem: {os.path.basename(path)} | Distância: {dist:.4f}")

def selecionar_imagem():
    Tk().withdraw()
    return askopenfilename(title="Selecione uma imagem", filetypes=[("Imagens", "*.jpg;*.jpeg;*.png;*.bmp")])

if __name__ == "__main__":
    imagem_usuario_path = selecionar_imagem()
    if imagem_usuario_path:
        # Use the absolute path provided by the user
        base_dir = r"C:\Users\Admin\Desktop\projeto economia\train"
        comparar_com_base(imagem_usuario_path, base_dir)


c:\Users\Admin\Desktop\projeto economia\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\Desktop\projeto economia\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Diretório de trabalho atual: c:\Users\Admin\Desktop\projeto economia\.ipynb_checkpoints
Usando diretório base: C:\Users\Admin\Desktop\projeto economia\train
Extraindo embedding da imagem do usuário: C:/Users/Admin/Downloads/1453293531982.jpg
Total de imagens na base: 832
Processadas 100 imagens...
Processadas 200 imagens...
Processadas 300 imagens...
Processadas 400 imagens...


KeyboardInterrupt: 

In [18]:
import pandas as pd
import random

# Path to your Excel file
excel_file_path = r'C:\Users\Admin\Desktop\projeto economia\Healthy_Food_Plates_100.xlsx'

# Read the Excel file
df = pd.read_excel(excel_file_path)

# Columns corresponding to Excel columns A to F
columns = ['Plate Name', 'Protein (g)', 'Calories', 'Fat (g)', 'Carbs (g)', 'Fiber (g)']

# Generate a random row number between 2 and 100 (inclusive)
random_row = random.randint(2, 100)

# Since pandas uses zero-based indexing, row 2 corresponds to index 1
row_index = random_row - 1

print(f"Reading row {random_row} from columns {columns}")

# Extract and print the single row as a DataFrame
data_row = df.loc[[row_index], columns]
print(data_row)

Reading row 39 from columns ['Plate Name', 'Protein (g)', 'Calories', 'Fat (g)', 'Carbs (g)', 'Fiber (g)']
                                           Plate Name  Protein (g)  Calories  \
38  Quinoa and Edamame with Steamed Broccoli and R...         22.0       385   

    Fat (g)  Carbs (g)  Fiber (g)  
38      6.7         64       15.8  
